# Transformers 코드 필사 

[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)   

"Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현하는 코드 필사해보기 


Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="center"/>



In [1]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

### Frame
- `EncoderDecoder`


- `Generator`

In [2]:
class EncoderDecoder(nn.Module):
    """
    Encoder-Decoder 뼈대 모델
    """
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder # encoder layer
        self.decoder = decoder # decoder layer
        self.src_embed = src_embed # embedding layer before encoder 
        self.tgt_embed = tgt_embed # embedding layer before decoder 
        self.generator = generator # classification (prediction) layer after decoder (FFNN + softmax)
    
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        src와 tgt입력을 각각 인코더, 디코더 처리해 그 결과를 반환
        """
        return self.decode(self.encode(src, src_mask),
                           src_mask,
                           tgt,
                           tgt_mask
                          )
    
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [3]:
class Generator(nn.Module):
    """
    vocab(총 단어의 수)에서 하나의 단어를 예측하는 linear + softmax 모델
    """
    
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`

- multi head attention & feed forward 두 개의 sublayer로 구성

  - Layer Normalization : 각 input의 feature들에 대한 평균과 분산을 구해서 각 미니배치에 있는 각 input을 정규화.
미니배치 샘플간 의존관계 없음, 배치사이즈에 영향을 많이 받지 않는 효과 
  - Residual Connection : 잔차 연결, 기존의 학습정보를 보존해가며 학습하기 위해 원래 값 x를 더해주는 것 

![스크린샷 2022-06-11 오후 10 19 01](https://user-images.githubusercontent.com/83392231/173189670-6c806ed0-09e5-45c7-81b0-f0bbbccd5f7d.png)



In [4]:
def clones(module, N):
    """module과 동일한 구조의 레이어를 N개 생성해 ModuleList에 담아 반환"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [5]:
class Encoder(nn.Module):
    """
    N개의 EncoderLayer를 쌓은 모델
    """
    
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size) # Custom Class
    
    
    def forward(self, x, mask):
        """
        입력값 x, mask를 순차적으로 EncoderLayer에 입력
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [6]:
class LayerNorm(nn.Module):
    """
    Layer Normalization = 각 입력값의 feature를 정규화
    """
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2
    

In [7]:
class SublayerConnection(nn.Module):
    """
    입력값을 순차적으로 
    1. layer normalization,
    2. sublayer,
    3. dropout,
    4. residual connection
    에 통과
    """
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        """
        residual connection을 반환
        """
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
class EncoderLayer(nn.Module):
    """
    2개의 sublayer로 구성된 인코더
    """
    
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder
- `Decoder`
- `DecoderLayer`
- 인코더에서 masked attention sublayer가 추가 된 형태 

![스크린샷 2022-06-11 오후 10 22 18](https://user-images.githubusercontent.com/83392231/173189768-500655a6-6539-4997-add0-6edcd58db815.png)


In [9]:
class Decoder(nn.Module):
    """
    N개의 DecoderLayer를 쌓은 모델
    """
    
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        """
        입력값 x, tgt_mask와 encoder에서 전달 받은 memory, src_mask를 순차적으로 DecoderLayer에 입력 
        """
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [10]:
class DecoderLayer(nn.Module):
    """
    3개의 sublayer로 구성된 디코더
    """
    
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x : self.self_attn(x,x,x,tgt_mask))
        x = self.sublayer[1](x, lambda x : self.self_attn(x,m,m,src_mask))
        return self.sublayer[2](x, self.feed_forward)
    

### Sublayer
- `attention` 함수

![스크린샷 2022-06-11 오후 9 44 51](https://user-images.githubusercontent.com/83392231/173188513-6c1d9855-eab6-4e01-843e-17a71b48a02e.png)

$$ \sqrt{d_k} : 차원의\ 제곱근으로\ score가\ 너무\ 커져\ gradient가\ 작아지는\ 현상을\ 방지하기\ 위해\ 나눠줌 $$ 

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

In [11]:
def attention(query, key, value, mask=None, dropout=None):
    
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) 
    
    # mask된 token 위치의 score 값을 아주 작은 값인 -1e9로 대체함 (masked token은 attention 계산을 하지 않기 위함)
    if mask is not None:
        scores= scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    attention = torch.matmul(p_attn, value)
    return  attention, p_attn

- `MultiHeadedAttention`

헤드를 여러개 사용하는 어텐션 구조. 어텐션 결과의 정확도를 높이기 위해서 여러개의 헤드를 사용한후 그 결과값을 더하는 형태로 진행. 

$$ Multi-head\ attention = concatenate(Z_1, Z_2, ... Z_8)W_0  $$

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [12]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h # multi headed attention은 d_model을 head 개수로 나눠 dimesion reduction을 함.
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(dropout)
    
    
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(dim=1) # 새로운 dimension을 (axis 1 앞에) 추가
        nbatches = query.size(0)
        
        # query, key, value를 각각 서로 다른 linear layer에 통과시켜(=linear projection) 얻은 값은 다시 query, key, value 변수에 할당
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
          for l, x in zip(self.linears, (query,key,value))]
        
        # attention 적용
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # 8개 head의 attention을 concatenate하여 마지막 linear layer에 통과
        x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
            

- `PositionwiseFeedForward`



In [13]:
class PositionwiseFeedForward(nn.Module):
    
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`


In [14]:
class Embeddings(nn.Module):
    """
    입력값 x (연속적인 토큰)를 지정된 vocab 사이즈의 lookup 테이블에서 d_model 사이즈의 엠베딩으로 변환
    """
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

트랜스포머는 입력문장에서 단어의 순서를 고려하지 않음 

단어의 위치 정보를 제공하기 위해 위치별로 특정 패턴을 따르는 포지셔널 인코딩 추가 

![스크린샷 2022-06-11 오후 10 42 17](https://user-images.githubusercontent.com/83392231/173190443-cc48a870-a958-4dba-a475-eb1d22c6ae20.png)


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  



In [15]:
class PositionalEncoding(nn.Module):
    """
    입력값 x (엠베딩 된 3차원 텐서 (nbatches, max_len, d_model))에 positional encoding을 더해 반환
    """
    
    def __init__(self, d_model, dropout, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0,max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                        requires_grad=False)
        return self.dropout(x)
        

### Finally Build Model


In [16]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(
            EncoderLayer(d_model, c(attn), c(ff), dropout),
            N
        ),
        Decoder(
            DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout),
            N
        ),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )
    
   # 파라미터들을 xavier_uniform 분포로 초기화 
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model
            

In [17]:
model = make_model(10,10)

In [18]:
for name, value in model.state_dict().items():
    print(f"{name} shape: {value.shape}")

encoder.layers.0.self_attn.linears.0.weight shape: torch.Size([512, 512])
encoder.layers.0.self_attn.linears.0.bias shape: torch.Size([512])
encoder.layers.0.self_attn.linears.1.weight shape: torch.Size([512, 512])
encoder.layers.0.self_attn.linears.1.bias shape: torch.Size([512])
encoder.layers.0.self_attn.linears.2.weight shape: torch.Size([512, 512])
encoder.layers.0.self_attn.linears.2.bias shape: torch.Size([512])
encoder.layers.0.self_attn.linears.3.weight shape: torch.Size([512, 512])
encoder.layers.0.self_attn.linears.3.bias shape: torch.Size([512])
encoder.layers.0.feed_forward.w_1.weight shape: torch.Size([2048, 512])
encoder.layers.0.feed_forward.w_1.bias shape: torch.Size([2048])
encoder.layers.0.feed_forward.w_2.weight shape: torch.Size([512, 2048])
encoder.layers.0.feed_forward.w_2.bias shape: torch.Size([512])
encoder.layers.0.sublayer.0.norm.a_2 shape: torch.Size([512])
encoder.layers.0.sublayer.0.norm.b_2 shape: torch.Size([512])
encoder.layers.0.sublayer.1.norm.a_2 sh